In [23]:
import ccxt
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

binance = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '5m'
limit = 1000

since_dt = datetime.now(timezone.utc) - timedelta(days=4*3 + 3) # 4 weeks (+3 to avoid weekends if i fetch index, like the dxy, where they dont work on weekends, and i just remove where dxy is Nan)
since = int(since_dt.timestamp() * 1000) # second in milliseconds

all_candles = []

while True:
    candles = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit) # binance api from ccxt
    if not candles:
        break
    all_candles += candles

    since = candles[-1][0] + 1 # update since to last candle timestamp + 1 ms
    time.sleep(0.5)  # avoid hitting rate limits with api

# Convert to DataFrame (table)
df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [24]:
import yfinance as yf

dxy_daily = yf.download(
    'DX-Y.NYB',
    start=df['timestamp'].min().date() - pd.Timedelta(days=3),  # to avoid weekends
    end=df['timestamp'].max().date() + pd.Timedelta(days=3),
    interval='1d',
    progress=False,
    auto_adjust=False
)

# Reset index to move 'Date' to a column
dxy_daily = dxy_daily.reset_index(drop=False)

# Because its multilevel, make it one level (like df)
if isinstance(dxy_daily.columns, pd.MultiIndex):
    dxy_daily.columns = [col[0] for col in dxy_daily.columns]

dxy_daily.rename(columns={'Date': 'date'}, inplace=True)
dxy_daily.rename(columns={'Close': 'dxy_close'}, inplace=True)

dxy_daily["date"] = pd.to_datetime(dxy_daily["date"]).dt.date
df["date"] = pd.to_datetime(df["timestamp"]).dt.date

print(dxy_daily)

df = pd.merge(df, dxy_daily[["date", "dxy_close"]], on="date", how="left")

df['dxy_close'] = df['dxy_close'].ffill()
df = df.dropna(subset=['dxy_close']).reset_index(drop=True)
df.rename(columns={'dxy_close': 'dxy'}, inplace=True)

print(dxy_daily)

          date  Adj Close  dxy_close       High        Low       Open  Volume
0   2025-09-12  97.550003  97.550003  97.860001  97.489998  97.519997       0
1   2025-09-15  97.300003  97.300003  97.699997  97.269997  97.629997       0
2   2025-09-16  96.629997  96.629997  97.389999  96.559998  97.339996       0
3   2025-09-17  96.870003  96.870003  97.070000  96.220001  96.620003       0
4   2025-09-18  97.349998  97.349998  97.599998  96.839996  96.930000       0
5   2025-09-19  97.639999  97.639999  97.809998  97.269997  97.339996       0
6   2025-09-22  97.330002  97.330002  97.820000  97.309998  97.650002       0
7   2025-09-23  97.260002  97.260002  97.459999  97.199997  97.320000       0
8   2025-09-24  97.870003  97.870003  97.919998  97.220001  97.239998       0
9   2025-09-25  98.550003  98.550003  98.610001  97.739998  97.820000       0
10  2025-09-26  98.150002  98.150002  98.529999  98.129997  98.500000       0
11  2025-09-29  97.910004  97.910004  98.180000  97.769997  98.1

In [25]:
import ta
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Short-term (~30min)
df['rsi_6'] = RSIIndicator(df['close'], window=6).rsi()

# Classic (~1hour)
df['rsi_12'] = RSIIndicator(df['close'], window=12).rsi()

macd_indicator = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)

df['macd'] = macd_indicator.macd()           # MACD line
df['macd_signal'] = macd_indicator.macd_signal() # Signal line
df['macd_hist'] = macd_indicator.macd_diff()    # Histogram



In [26]:

# df['ema_9'] = df['close'].ewm(span=9, adjust=False).mean()

df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()

df['sma_50'] = df['close'].rolling(window=50).mean()

df.dropna(subset=['ema_21', 'sma_50'], inplace=True)

df['ema/sma crossover'] = df['ema_21'] - df['sma_50']

df['trends with sma'] = df['close'] / df['sma_50'] - 1

In [27]:
from utils import buy_col, sell_col, define_activity

df['next_high'] = df['high'].shift(-1)
df['next_low'] = df['low'].shift(-1)

df['price change'] = np.log(df['close'] / df['close'].shift(1))
df.dropna(subset=['price change'], inplace=True)

df['buy_index'] = df.apply(buy_col, axis=1)
df['sell_index'] = df.apply(sell_col, axis=1)

df['signal'] = df['buy_index'] - df['sell_index']
scale = df['signal'].abs().median()
df['signal_scaled'] = 0.5 * (np.tanh( (df['buy_index'] - df['sell_index']) / scale ) + 1)

df['activity'] = df.apply(define_activity, axis=1)

df.dropna(subset=['buy_index','sell_index'], inplace=True)


In [28]:
print(df)

# export_df = df[['dxy', 'rsi_6', 'rsi_12', 'ema_21', 'sma_50', 'activity']]
export_df = df.drop(columns=['timestamp', 'volume', 'date', 'next_high', 'next_low', 'buy_index', 'sell_index', 'signal', 'signal_scaled'])
print(export_df)

%store export_df

               timestamp       open       high        low      close  \
50   2025-09-15 12:20:00  114959.29  115017.80  114916.33  114970.00   
51   2025-09-15 12:25:00  114970.00  114983.02  114789.00  114794.01   
52   2025-09-15 12:30:00  114794.01  114880.20  114789.00  114872.01   
53   2025-09-15 12:35:00  114872.00  114885.00  114848.33  114852.01   
54   2025-09-15 12:40:00  114852.01  114888.80  114830.98  114878.24   
...                  ...        ...        ...        ...        ...   
4314 2025-09-30 07:40:00  113658.26  113701.37  113596.37  113635.77   
4315 2025-09-30 07:45:00  113635.76  113755.71  113585.00  113658.43   
4316 2025-09-30 07:50:00  113658.42  113800.00  113638.92  113740.11   
4317 2025-09-30 07:55:00  113740.12  113787.56  113657.20  113699.23   
4318 2025-09-30 08:00:00  113699.24  113741.37  113646.01  113666.22   

         volume        date        dxy      rsi_6     rsi_12  ...  \
50    108.52415  2025-09-15  97.300003  48.980304  49.986903  ... 